In [1]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
def show_image(img):
    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def calibrate_camera(imgs_path, nx, ny):
    images = glob.glob(imgs_path + "/cal*.jpg")
    # images = os.listdir(imgs_path)
    objpoints = [] 
    imgpoints = []

        
    for fname in images:
        img = cv2.imread(fname)
        objp = np.zeros((nx*ny, 3), np.float32)
        objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret is True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return objpoints, imgpoints

In [4]:
def get_undist_img(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [5]:
objpoints, imgpoints = calibrate_camera("camera_cal/", 9, 6)

In [6]:
chess_img = cv2.imread("camera_cal/calibration1.jpg")

In [7]:
undist = get_undist_img(chess_img, objpoints, imgpoints)

In [8]:
# cv2.imwrite("../notes/images/chess_img.jpg", chess_img)
# cv2.imwrite("../notes/images/undistorted_chess.jpg", undistorted_img)

In [11]:
def abs_sobel_thresh(img, sobel_kernel=3, orient='x',  thresh=(100,255)):
    if orient=='x':
        sobel = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    if orient=='y':
        sobel = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobelabs = np.abs(sobel)
    scaledsobel = np.uint8(255*sobelabs/np.max(sobelabs))
    sbinary = np.zeros_like(scaledsobel)
    sbinary[(scaledsobel >= thresh[0]) & (scaledsobel <= thresh[1])] = 255
    return sbinary

In [6]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    magnitude = np.sqrt((sobelx)**2 + (sobely)**2)
    scaledsobel = np.uint8(255*magnitude/np.max(magnitude))
    sbinary = np.zeros_like(scaledsobel)
    sbinary[(scaledsobel >= mag_thresh[0]) & (scaledsobel <= mag_thresh[1])] = 255
    return sbinary

In [7]:
def dir_threshold(img, sobel_kernel=3, thresh=(0.7, 1.3)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.abs(sobelx)
    abs_sobely = np.abs(sobely)
    direction = np.arctan2(abs_sobely, abs_sobelx)
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 255
    return binary_output

In [8]:
img = cv2.imread("test_images_new/502.jpg")

In [9]:
show_image(img)

In [10]:
undist = get_undist_img(img, objpoints, imgpoints)

In [11]:
#cv2.imwrite("../notes/images/undist.jpg", undist)

In [12]:
def perspective_transform(undist_img):
    offset = 50
    src = np.float32([[558, 462],
                  [740, 462],
                  [1230, undist_img.shape[0]],
                  [undist_img.shape[1] / 7, undist_img.shape[0]]
                 ])

    dst = np.float32([[undist_img.shape[1] / 9 , offset],
                 [undist_img.shape[1] + offset, offset],
                 [undist_img.shape[1] - offset, undist_img.shape[0]],
                 [undist_img.shape[1] / 6, undist_img.shape[0]]])

    Minv = cv2.getPerspectiveTransform(dst, src)
    M = cv2.getPerspectiveTransform(src, dst)

    warped = cv2.warpPerspective(undist_img, M, undist_img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [14]:
show_image(undist)

In [13]:
warped, M, Minv = perspective_transform(undist)

In [14]:
show_image(warped)

In [51]:
#cv2.imwrite("../notes/images/warped_straightlines.jpg", warped)

In [15]:
def color_pipeline(img, l_thresh = (120, 255), s_thresh=(120, 255), sx_thresh=(20, 255)):
    img = np.copy(img)
    
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[...,2]
    l_channel = hls[...,1]
    h_channel = hls[...,0]
    
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Sobel x with l_channel for shadow in the road
    sobelx_l = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0)
    abs_sobelx_l = np.absolute(sobelx_l)
    scaled_sobel_l = np.uint8(255*abs_sobelx_l/np.max(abs_sobelx_l))
    
    # sobelx in s channel
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255

    # sobelx in l channel
    sxbinary_l = np.zeros_like(scaled_sobel_l)
    sxbinary_l[(scaled_sobel_l >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255
    
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 255

    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 255
    
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) 


    combined_binary = np.zeros_like(sxbinary)
    combined_binary[ ((sxbinary == 255) | (sxbinary_l == 255)) | ((s_binary == 255) & (l_binary == 255)) ] = 255
    
    gradient_combined = np.zeros_like(sxbinary)
    gradient_combined[((sxbinary==255) | (sxbinary_l) == 255)] =255
    
    
    color_combined = np.zeros_like(sxbinary)
    color_combined[((s_binary==255) & (l_binary==255))] = 255
    return combined_binary

In [16]:
combined_binary  = color_pipeline(warped)

In [58]:
show_image(combined_binary)

In [55]:
#cv2.imwrite("../notes/images/combined_binary.jpg", combined_binary)

In [48]:
def find_lane_pixels(binary_warped):

    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])

    rightx_base = np.argmax(histogram[midpoint:]) + midpoint 

    nwindows = 9
    margin = 50
    minpix = 50

    window_height = np.int(binary_warped.shape[0]//nwindows)

    nonzero = binary_warped.nonzero()

    nonzeroy = np.array(nonzero[0]) 
    nonzerox = np.array(nonzero[1]) 

    leftx_current = leftx_base
    rightx_current = rightx_base


    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
   
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin


        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0, 255, 0), 3)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0, 255, 0), 3)

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

In [38]:
leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)

In [19]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fitx, right_fitx, ploty

In [20]:
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

In [21]:
def old_search_around_poly(binary_warped, left_fit, right_fit):

    margin = 70

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
                      (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) &
                      (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin,
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin,
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))


    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    left_pts = np.vstack((left_fitx, ploty)).astype(np.int32).T
    right_pts = np.vstack((right_fitx, ploty)).astype(np.int32).T
    cv2.polylines(result, [left_pts], False, (0, 255, 255), 10)
    cv2.polylines(result, [right_pts], False, (0, 255, 255), 10)

    return result, left_fitx, right_fitx, ploty

In [22]:
result, left_fitx, right_fitx, ploty = old_search_around_poly(combined_binary,
                                                             left_fit, right_fit)

In [28]:
show_image(result)

In [23]:
def search_around_poly(undist, binary_warped, Minv, left_fit, right_fit):


    margin = 70


    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
                      (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) &
                      (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))


    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]


    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)


    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    lanes_img = np.zeros_like(out_img)

    
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    pts = np.hstack((pts_left, pts_right))
    
    cv2.fillPoly(lanes_img, np.int_([pts]), (0, 55, 0))
    
    
    ## Yellow fitted lines. To see if this is usefull to 
    # get the position of the car
    left_pts = np.vstack((left_fitx, ploty)).astype(np.int32).T
    right_pts = np.vstack((right_fitx, ploty)).astype(np.int32).T
#     cv2.polylines(result, [left_pts], False, (255, 255, 0), 10)
#     cv2.polylines(result, [right_pts], False, (255, 255, 0), 10)
    
    
    # Color in left and right line pixels
    lanes_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [0, 0, 255]
    lanes_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [255, 0, 0]
    
    newwarp = cv2.warpPerspective(lanes_img, Minv, (lanes_img.shape[1], lanes_img.shape[0]))
    
    
    output = cv2.addWeighted(undist, 0.8, newwarp, 1, 0)
    
    
    

    return output, left_pts, right_pts

In [24]:
output, left_pts, right_pts = search_around_poly(undist, combined_binary, Minv, left_fit, right_fit)

In [51]:
left_pts

array([[573,   0],
       [572,   1],
       [572,   2],
       ..., 
       [231, 717],
       [231, 718],
       [231, 719]], dtype=int32)

In [25]:
show_image(output)

# Measure Curvature

In [26]:
def measure_curvature_pixels():
    leftx, lefty, rightx, righty, _ = find_lane_pixels(combined_binary)


    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, combined_binary.shape[0]-1, combined_binary.shape[0] )
    
    y_eval = np.max(ploty)
    
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

In [27]:
left_curverad, right_curverad = measure_curvature_pixels()

In [28]:
def measure_curvature_real(img):
    ym_per_pix = 30/720
    xm_per_pix = 3.7/900
    
    leftx, lefty, rightx, righty, _ = find_lane_pixels(img)
    
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    
    y_eval = np.max(ploty)
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

In [29]:
left_curverad, right_curverad = measure_curvature_real(combined_binary)

In [30]:
# def proc_pipeline_tmp(img, objpoints, imgpoints):

#     undist = get_undist_img(img, objpoints, imgpoints)
#     warped_img, M, Minv = perspective_transform(undist)
#     combined_binary = color_pipeline(warped_img)
#     leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)
    
#     return out_img

In [50]:
def compute_location(warped, left_pts, right_pts):
    xm_per_pix = 3.7/900
    dist_right = right_pts[-1][0] 
    dist_left = left_pts[-1][0]
    main_dist = (dist_right+dist_left)/2 - warped.shape[1]/2
    main_dist = main_dist*xm_per_pix
    
    

    return main_dist

(a+b)/2 - W/2, where W is the width of the birds-eye view image. Of course, this has to be converted to meters with the help of xm_per_pix.

In [39]:
def proc_pipeline_video(img, objpoints, imgpoints):

    undist = get_undist_img(img, objpoints, imgpoints)
    warped_img, M, Minv = perspective_transform(undist)
    combined_binary = color_pipeline(warped_img)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    


    left_curverad, right_curverad = measure_curvature_real(combined_binary)
    
    output, left_pts, right_pts = search_around_poly(undist, combined_binary, Minv, left_fit, right_fit)
    
    main_dist = compute_location(warped_img, left_pts, right_pts)
    
    
    cv2.putText(output, "Radius: " + str(int(left_curverad)) + "(m)", 
            (output.shape[0] // 2, output.shape[1] // 8), 
            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255))
        
    cv2.putText(output, "Vehicle is : " + str(round(main_dist, 2)) + " (m) to the left",
            (output.shape[0] // 2, output.shape[1] // 6),
        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255))
    
    return output

https://answers.opencv.org/question/200077/combine-several-videos-in-the-same-window-python/

In [40]:
def resize_frame(frame, factor=2):
    resized = cv2.resize(frame, (frame.shape[1] // factor, frame.shape[0] // factor),
                         fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    return resized

In [42]:
process_video("test_videos/project_video.mp4")

In [42]:
# ## Several images

# cap1 = cv2.VideoCapture("test_videos/project_video.mp4")


# while (cap1.isOpened()):
#     ret, frame = cap1.read()
#     if ret:
#         combined = proc_pipeline_tmp(frame, objpoints, imgpoints)
#         full = proc_pipeline_video(frame, objpoints, imgpoints)
#         pro = proc_pipeline_tmp(frame, objpoints, imgpoints) 
#         pro2 = proc_pipeline_tmp(frame, objpoints, imgpoints)

#         combined_res = resize_frame(combined)
#         full_res = resize_frame(full)
#         pro_res = resize_frame(pro)
#         pro2_res = resize_frame(pro2)


#         both = np.concatenate((combined_res, full_res), axis=1)
#         both2 = np.concatenate((pro_res, pro2_res), axis=1)
#         all_4 = np.concatenate((both, both2), axis=0)
#         cv2.imshow("frame", all_4)
        
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     else:
#         break
# # Release everything if job is finished
# cap1.release()
# cv2.destroyAllWindows()

http://opencv-users.1802565.n2.nabble.com/Combining-Multiple-videos-td2188079.html

In [252]:
def write_img_from_video(video_path):
    count = 0
    video_capture = cv2.VideoCapture(video_path)
    while (video_capture.isOpened()):
        ret, frame = video_capture.read()
        if ret:
            cv2.putText(frame, "{}".format(count), (frame.shape[0] // 2, frame.shape[1] // 5), 
            cv2.FONT_HERSHEY_COMPLEX_SMALL, 4, (255, 255, 255))
            
            undist = get_undist_img(frame, objpoints, imgpoints)
            warped_img, M, Minv = perspective_transform(undist)
            combined_binary = color_pipeline(warped_img)
            
            result, _, _, _ = old_search_around_poly(combined_binary,
                                                             left_fit, right_fit)
            
            output, _, _ = search_around_poly(undist, combined_binary, Minv, left_fit, right_fit)
            
            
            cv2.imshow('frame',frame)
            
            cv2.imwrite('test_images_new/' + str(count) + ".jpg", frame)
            cv2.imwrite("test_images_new/" + str(count) + "warped.jpg", warped_img)
            cv2.imwrite("test_images_new/" + str(count) + "combined_binary.jpg", combined_binary)
            cv2.imwrite("test_images_new/" + str(count) + "output.jpg", output)
            cv2.imwrite("test_images_new/" + str(count) + "fitted.jpg", result)
            
            
            video_capture.set(cv2.CAP_PROP_POS_MSEC, (count*50))
            
            count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    # Release everything if job is finished
    video_capture.release()
    cv2.destroyAllWindows()

In [253]:
write_img_from_video("test_videos/project_video.mp4")

In [49]:
white_output = 'test_videos_output/project_video_output.mp4'

clip1 = VideoFileClip("test_videos/project_video.mp4")
white_clip = clip1.fl_image(lambda img: proc_pipeline_video(img, objpoints, imgpoints)) 
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/project_video_output.mp4.
Moviepy - Writing video test_videos_output/project_video_output.mp4




t:  18%|█▊        | 228/1260 [02:44<12:31,  1.37it/s, now=None]

KeyboardInterrupt: 